# World Wide Products Inc.
Data source : https://www.kaggle.com/felixzhao/productdemandforecasting#Historical%20Product%20Demand.csv


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from fbprophet import Prophet

ModuleNotFoundError: No module named 'fbprophet'

### Load data

Convert 'Data' into date type while input.

In [ ]:
df = pd.read_csv("../data/Historical Product Demand.csv",parse_dates=['Date'], infer_datetime_format=True)

# Feature explore

### Check null data and drop

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.head()

### Convert Order_demand into numeric type

In [ ]:
df['Order_Demand'] = df['Order_Demand'].map(lambda x: x.lstrip('(').rstrip(')'))
df['Order_Demand'] = pd.to_numeric(df['Order_Demand'])

In [ ]:
df.info()

There are 33 product categories and 2160 products.

In [ ]:
ax = df.groupby('Product_Category')['Product_Code'].nunique().plot(kind='bar',
                                    figsize=(20,5),
                                    title="Number of Products in each Product Category")
for i in ax.patches:
    ax.text(i.get_x(), i.get_height(), str(i.get_height()))

In [ ]:
ax = df.groupby('Product_Category')['Order_Demand'].sum().plot(kind='bar',
                                    figsize=(20,5),
                                    title="Number of Demands in each Product Category")
for i in ax.patches:
    ax.text(i.get_x(), i.get_height(), str(i.get_height()))

As the above figs showed,category 019 is the most frequent. Now we will select products in Category 19 to do some experiment.

In [ ]:
product_num = df[df['Product_Category']== 'Category_019'].groupby(['Product_Code']).size().reset_index(name="rows")
product_num.sort_values(['rows'],ascending=False).head(10)

### Look into a specific product

In [ ]:
Product_1 = df[df["Product_Code"]=="Product_1295"]
Product_1.head()

In [ ]:
Product_1 = Product_1.groupby('Date')['Order_Demand'].sum().reset_index()
Product_1.head()

In [ ]:
Product_1.pivot_table('Order_Demand', index='Date').plot(figsize=(20,6))
plt.ylabel('Demand');

### The demands on months
#### We need index data with time

In [ ]:
Product_1 = Product_1.set_index('Date')
Product_1.index

Check the date range

In [ ]:
Product_1.index.min(), Product_1.index.max()

In [ ]:
Product_1 = Product_1["Order_Demand"].resample('MS').sum()
Product_1.head()

In [ ]:
Product_1.plot(figsize=(20,6))
plt.ylabel('Demand');

The products of the same category

In [ ]:
Product_2 = df[df["Product_Code"]=="Product_1359"].groupby('Date')['Order_Demand'].sum().reset_index()
Product_2 = Product_2.set_index('Date')

In [ ]:
Product_2 = Product_2["Order_Demand"].resample('MS').sum()

In [ ]:
Product_2.index.min(), Product_2.index.max()

We drop the data in 2017 as the data is imcomplete for 2017

In [ ]:
Product_2 = Product_2.drop(pd.to_datetime('2017-01-01'))

In [ ]:
product1 = pd.DataFrame({'Date':Product_1.index, 'Order_Demand':Product_1.values})
product2 = pd.DataFrame({'Date': Product_2.index, 'Order_Demand': Product_2.values})
MergeProduct = product1.merge(product2, how='inner', on='Date')
MergeProduct.head()

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(MergeProduct['Date'], MergeProduct['Order_Demand_x'], 'b-', label = 'Product_1295')
plt.plot(MergeProduct['Date'], MergeProduct['Order_Demand_y'], 'r-', label = 'Product_1359')
plt.xlabel('Date'); plt.ylabel('Demand'); plt.title('Demand of Product_1295 and Product_1359')
plt.grid()
plt.legend();

# Forecast
Forecasting with Facebook Prophet. Link:https://github.com/facebook/prophet
Need to install fbprophet

In [ ]:
product1_model = Prophet(interval_width=0.95, weekly_seasonality=True)
product1_model.fit(product1)

### Divide the date column into year, month and day

In [ ]:
# Convert the series' types
# df['year'] = df['Date'].dt.year.astype(int)
# df['month'] = df['Date'].dt.month.astype(int)
# df['day'] = df['Date'].dt.day.astype(int)